In [ ]:
# 한국어를 이용한 감성 분류
# 이진분류 문제 => 리뷰내용이 긍정인지 부정인지 학습한 후 예측!
# 기존에 배웠던 Token, Tokenizer, Padding,  vacabulary, One-Hot, Embedding
# 이런 개념들이 어떻게 사용되는지 코드로 확인해 보면 될 듯 해요!

In [8]:
# 필요한 module부터 불러와요!
%reset -f

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import tensorflow as tf

In [ ]:
# 네이버에서 제공하는 네이버 영화 리뷰 데이터셋을 다운로드해요.
train_file = tf.keras.utils.get_file(
    cache_dir='./data', # 다운로드 경로
    fname='ratings_train.txt',
    origin='http://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',
    extract=True
)

train_path = os.path.join('./data/datasets', 'ratings_train.txt')
train_df = pd.read_csv(train_path, sep='\t')
display(train_df) # 리뷰 개수가 15만개에요!

14628807/14628807 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step 


/tmp/ipykernel_4814/942972367.py:2: UserWarning: Could not extract archive.
  train_file = tf.keras.utils.get_file(


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
# 네이버에서 제공하는 네이버 영화 리뷰 데이터셋을 다운로드해요.
test_file = tf.keras.utils.get_file(
    cache_dir='./data', # 다운로드 경로
    fname='ratings_test.txt',
    origin='http://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt',
    extract=True
)

test_path = os.path.join('./data/datasets', 'ratings_test.txt')
test_df = pd.read_csv(test_path, sep='\t')
display(test_df) # 테스트용 리뷰 개수가 5만개에요!

4893335/4893335 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/tmp/ipykernel_4814/2378423992.py:2: UserWarning: Could not extract archive.
  test_file = tf.keras.utils.get_file(


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [ ]:
# 간단하게 데이터 특성에 대해서 알아보아요!
# 이진분류를 하고 있기 때문에 적어도 긍정과 부정이 1:1 비율로 있어야
# 학습이 잘 이루어질 것 같아요!
cnt = train_df['label'].value_counts()
print(cnt) # 데이터 불균형 문제는 없을 것 같아요!

print(train_df.isnull().sum()) # 결측치가 5개가 있어요.
                               # 삭제를 해야할 것 같아요!

label
0    75173
1    74827
Name: count, dtype: int64
id          0
document    5
label       0
dtype: int64


In [ ]:
# Token을 구별해서 이 Token으로 vocabulary를 구성해야 해요!
# 그런데 한국어는 이 Token을 분리해 내는 작업 자체가 어려워요!
# => 형태소 분석으로
# 형태소 분석은 어떻게 하면 되나요?
# => 라이브러리가 있어요!
# 그럼 가장 많이 사용되는 형태소 분석기는 어떤 것들이 있나요?
# 1. Mecab - 가장 널리 사용되요! C++기반 빠르고 정확해요. 실무에서 주로 사용.
#            설치가 약간 복잡.
# 2. Okt(Open Korean Text Processor) - 학습용으로 사용되요! 설치가 간단.
# 3. Kiwi - 딥러닝 기반으로 띄어쓰기 오류에 강해요!

# 우리는 Mecab을 설치해서 사용할 거에요!
# Ubuntu Linux에 설치하는 과정

# 필요한 package 설치
# sudo apt update
# sudo apt install -y make curl git build-essential autoconf automake libtool

# Mecab 설치
# sudo apt install -y mecab libmecab-dev mecab-ipadic-utf8

# mecab-ko-dic 다운로드
# cd /tmp
# wget https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.1.1-20180720.tar.gz
# tar zxfv mecab-ko-dic-2.1.1-20180720.tar.gz
# cd mecab-ko-dic-2.1.1-20180720

# 사전 컴파일 및 설치
# ./autogen.sh
# ./configure
# make
# sudo make install

# 설치 경로 확인
# mecab-config --dicdir

# 설정파일 확인
# sudo vi /usr/local/etc/mecabrc

# 이때 Vim에서 해당 내용 수정
'''
dicdir = /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ko-dic -> 확인한 경로로 수정
userdic =
'''

In [ ]:
# Mecab 설치가 다 되었으면 -> 확인을 한 후에
# 이 Mecab을 Python에서 사용할거에요!
# 이 작업을 수행해 줄 Python module을 설치해야 해요!
# pip install mecab-python3
# pip install konlpy

In [9]:
# 코드상에서 형태소 분석을 해 보아요!
from konlpy.tag import Mecab, Okt

dicpath='/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ko-dic'

okt = Okt()
mecab = Mecab(dicpath=dicpath)

text = '이것은 소리없는 아우성. 저 푸른 해원을 향하여 흔드는 노스텔지어의 손수건'

print(okt.morphs(text))
print(mecab.morphs(text))

['이', '것', '은', '소리', '없는', '아우성', '.', '저', '푸른', '해원', '을', '향', '하여', '흔드는', '노스', '텔', '지', '어의', '손수건']
['이것', '은', '소리', '없', '는', '아우성', '.', '저', '푸른', '해원', '을', '향하', '여', '흔드', '는', '노', '스텔', '지어', '의', '손수건']


In [12]:
# 그럼 처음부터 데이터 전처리까지 순차적으로 한번 코드작업을 수행해 보아요!
%reset -f

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import tensorflow as tf

# 네이버에서 제공하는 네이버 영화 리뷰 데이터셋을 다운로드 해요.
train_file = tf.keras.utils.get_file(
    cache_dir='./data', # 다운로드 경로
    fname='ratings_train.txt', # 파일명
    origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',
    extract=True
)
train_path = os.path.join('./data/datasets', 'ratings_train.txt')
train_df = pd.read_csv(train_path, sep='\t')
display(train_df.head())

test_file = tf.keras.utils.get_file(
    cache_dir='./data',   # 다운로드 경로
    fname='ratings_test.txt',  # 파일명
    origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt',
    extract=True
)
test_path = os.path.join('./data/datasets', 'ratings_test.txt')
test_df = pd.read_csv(test_path, sep='\t')
display(test_df.head())

/tmp/ipykernel_65483/3779165594.py:12: UserWarning: Could not extract archive.
  train_file = tf.keras.utils.get_file(


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


/tmp/ipykernel_65483/3779165594.py:22: UserWarning: Could not extract archive.
  test_file = tf.keras.utils.get_file(


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
# 1. 데이터를 가져왔으니 이제 영어, 한글, 띄어쓰기(공백)만 남기고 나머지
#    특수문자를 제거 => 정규식을 이용해서 처리
train_df['document'] = train_df['document'].str.replace(r'[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]',
                                                        '',
                                                        regex=True)
display(train_df.head())

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [15]:
# 2. 결측치 5개 제거
train_df = train_df.dropna()
print(train_df.shape)

# 3. 불용어(stopword) 제거
#    관사, 전치사, 조사, 접속사 => 의미가 없는 단어를 지칭
#    한국어에는 정말 많은 불용어가 존재하는데 이 중 일부만 제거
#    불용어를 제거하는 함수를 이용해서 처리

from konlpy.tag import Okt, Mecab
from tqdm import tqdm

tqdm.pandas() # tqdm을 pandas에 부착(pandas의 dataframe을 처리할 때 progressbar 표시)

dicpath='/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ko-dic'
mecab = Mecab(dicpath=dicpath)
# okt = Okt()

# 형태소 분석 후 불용어를 제거해서 그 결과를 리턴하는 함수
def word_tokenization(text):

    stop_words = [
    # 조사
    '은', '는', '이', '가', '을', '를', '에', '에서', '에게', '한테', '으로', '로',
    '과', '와', '도', '만', '이나', '보다', '처럼', '까지', '부터', '라도', '마저',

    # 접속사
    '그리고', '그러나', '하지만', '그래서', '그러면', '그런데', '따라서', '혹은', '또는',

    # 의존명사/형식적 명사
    '수', '것', '거', '때', '중', '등', '뿐', '대로', '만큼', '따위',

    # 대명사/지시어
    '나', '너', '우리', '저희', '그', '이', '저', '그것', '이것', '저것', '자기',

    # 일반 동사/보조용언
    '되다', '하다', '있다', '없다', '이다', '아니다', '받다', '주다', '되어다', '같다', '되었다',

    # 감탄사/불필요 표현
    '아', '야', '어', '우와', '헐', '음', '응', '네', '예', '자', '좀', '요', '그냥', '또', '그래',

    # 빈도 높은 불필요 어휘
    '정말', '진짜', '너무', '매우', '아주', '항상', '더', '더욱', '계속', '이미', '이제',

    # 불용 보조 용언/어미 어절
    '것이다', '것이', '때문이다', '있습니다', '없습니다', '하는', '해서', '하였다', '했다', '하고', '하며', '하면서',

    # 기타 (프로젝트 목적 따라 제거 가능)
    '하지만', '그러나', '혹시', '그러면', '그럼', '혹은', '만약', '만일', '또는'
    ]

    return [word for word in mecab.morphs(text) if word not in stop_words]

data_train = train_df['document'].progress_apply(word_tokenization)
display(data_train.head())

(149995, 3)


  0%|          | 0/149995 [00:00<?, ?it/s]

100%|██████████| 149995/149995 [00:07<00:00, 19855.45it/s]


0                                     [빙, 짜증, 네요, 목소리]
1    [흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 조차, 가볍, 지, 않, 구나]
2                                  [재, 밓었다그래서보는것을추천한다]
3                [교도소, 이야기, 구먼, 솔직히, 재미, 없, 다, 평점, 조정]
4    [사이몬페그, 의, 익살, 스런, 연기, 돋보였, 던, 영화, 스파이더맨, 늙, 보...
Name: document, dtype: object

In [25]:
# 지금까지 진행하면... 형태소 분석과 불용어 처리까지 진행할 수 있어요!
# 이걸 이용해서 단어사전을 만들 수 있어요!
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_train) # 분리되어 있는 형태소가 결국 Token이 되고
                                   # 이 Token들을 이용해서 vocabulary 생성
# 총 단어 개수
print(len(tokenizer.word_index)) # 52185

# 이 5만개의 단어를 다 사용해서 문자열을 숫자 시퀀스로 변경할 건가요?
# 일반적으로 많은 빈도를 가지는 형태소(Token)만을 이용해서 문자열을 숫자 시퀀스로 변경
# 그럼 몇개가 적당할까요?
# print(tokenizer.word_counts)

def get_voca_size(threshold):
    cnt = 0
    for x in tokenizer.word_counts.values():
        if x > threshold:
            cnt = cnt + 1
    return cnt

voca_size = get_voca_size(5)
print(voca_size)

# 단어 사전을 생성할거에요!
tokenizer = Tokenizer(oov_token='<OOV>',
                      num_words=15000)

tokenizer.fit_on_texts(data_train)

# 단어 사전을 만들었으니 이걸 이용해서 문자열을 숫자의 시퀀스로 변경할 수 있어요!
data_train_seq = tokenizer.texts_to_sequences(data_train)
print(data_train_seq[0:3])

# 최종적으로 길이만 똑같이 만들어주면 우리 모델에 집어넣을 수 있는 형태가 완성될 것같아요!
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_data_train = pad_sequences(data_train_seq,
                             maxlen=75)
y_data_train = np.asarray(train_df['label'])

# 확인
print(x_data_train[0:1])

52185
13727
[[879, 187, 21, 663], [925, 449, 296, 593, 2, 89, 1512, 34, 754, 912, 11, 27, 331], [162, 1]]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 879
  187  21 663]]


In [ ]:
# 데이터는 준비가 끝났으니 이제 모델만 만들어주면 되요!
# 데이터 만들 때 one_hot encoding을 하지 않았어요 => Embedding을 사용
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

model = Sequential()

model.add(Embedding(input_dim=15001,
                    output_dim=128,
                    input_length=75))
model.add(LSTM(units=16,
               activation='tanh'))
model.add(Dense(units=1,
                activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 3가지 callback을 사용!
# 이 중 일단 EarlyStopping만 적용
es_callback = EarlyStopping(monitor='val_loss',
                            patience=5,
                            restore_best_weights=True,
                            verbose=1)

In [27]:
# 학습 진행
model.fit(x_data_train,
          y_data_train,
          epochs=100,
          batch_size=64,
          validation_split=0.2,
          callbacks=[es_callback],
          verbose=1)

Epoch 1/100


2025-07-09 17:51:51.232652: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 13ms/step - accuracy: 0.7948 - loss: 0.4372 - val_accuracy: 0.8516 - val_loss: 0.3416
Epoch 2/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.8753 - loss: 0.2898 - val_accuracy: 0.8562 - val_loss: 0.3343
Epoch 3/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.8999 - loss: 0.2387 - val_accuracy: 0.8543 - val_loss: 0.3479
Epoch 4/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9179 - loss: 0.2012 - val_accuracy: 0.8501 - val_loss: 0.3747
Epoch 5/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.9340 - loss: 0.1649 - val_accuracy: 0.8431 - val_loss: 0.4382
Epoch 6/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - accuracy: 0.9462 - loss: 0.1365 - val_accuracy: 0.8402 - val_loss: 0.4728
Epoch 7/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.9543 - loss: 0.1177 - val_accuracy: 0.8418 - val_loss: 0.5191
Epoch 7: early stopping
Restoring model weights from the end of the best epoch: 2.

In [28]:
# 학습이 끝난 다음
# 예측작업을 수행할 수 있어요!
review_sentences = ['내가 만들어도 이것보다는 잘 만들겠다',
                    '너무너무 재미있었어요. 감사합니다.',
                    '아...내돈..돈이 너무 아까워요!',
                    '이것도 영화라고 쯧쯧.. 스토리가 산으로 가요',
                    '감동과 재미가 있는 영화입니다.',
                    '너무너무 재미없다.. 잠와 죽는줄',
                    '너무너무 재미있다.. 잠이 확깨요']

# 이 Review를 모델에 넣어서 예측을 수행
# 데이터 전처리를 해야해요!
df = pd.DataFrame({'document': review_sentences})

# 1. 정규식 이용해서 특수문자 제거(한글, 영어, 공백 제외하고)
df['document'] = df['document'].str.replace(r'[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]',
                                            '',
                                            regex=True)
# 2. 결측치 처리 -> 할게 없어요!
# 3. 형태소 분석하고 불용어를 제거
data_predict = df['document'].progress_apply(word_tokenization)
# 4. 이미 만들어 놓은 vocabulary를 이용해서
#    우리 문장을 숫자로 변환
data_predict_seq = tokenizer.texts_to_sequences(data_predict)
# 5. 숫자로 변환이 됐으니 길이를 맞춰줘요. -> 75개!
x_data_predict = pad_sequences(data_predict_seq,
                               maxlen=75)

result = model.predict(x_data_predict)
print(result)

100%|██████████| 7/7 [00:00<00:00, 1315.83it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
[[0.1093169 ]
 [0.9931677 ]
 [0.0079077 ]
 [0.01173522]
 [0.9872938 ]
 [0.01043729]
 [0.72699064]]
